<a href="https://colab.research.google.com/github/dev-singh-chauhan/PharmaMAS_Project-/blob/main/PharmaMAS_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai openai langchain pandas reportlab streamlit python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of instructor to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.7/666.7 kB 21.6

In [2]:
# Colab cell 1: install required packages
!pip install --upgrade pip
# Try installing crewai; if it doesn't exist on pip, we attempted to provide a GitHub fallback.
# If crewai is unavailable, the notebook will automatically use fallback local agent classes.
!pip install crewai openai langchain pandas reportlab python-dotenv xlrd openpyxl --quiet || true


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [3]:
# Colab cell 2: set API key env var (use real key only if needed)
import os
from google.colab import userdata

# Use the Secrets Manager to store your API key
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# IMPORTANT: crewai requires OPENAI_API_KEY for OpenAI models.
# Try to get OPENAI_API_KEY from secrets, fall back to GOOGLE_API_KEY if not found.
try:
    openai_key = userdata.get("OPENAI_API_KEY")
except Exception:
    openai_key = None

if not openai_key:
    openai_key = os.environ.get("GOOGLE_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_key

# Set a default model name as well, often needed by libraries
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini" # Or another suitable model name
print("OPENAI_API_KEY and OPENAI_MODEL_NAME set from secrets/defaults.")

OPENAI_API_KEY and OPENAI_MODEL_NAME set from secrets/defaults.


In [4]:
# Colab cell 3: create folder structure and write mock JSON/CSV data
import os, json, csv

BASE = "/content/pharma_mas"
os.makedirs(BASE, exist_ok=True)
os.makedirs(os.path.join(BASE, "agents"), exist_ok=True)
os.makedirs(os.path.join(BASE, "crew"), exist_ok=True)
os.makedirs(os.path.join(BASE, "mock_data"), exist_ok=True)
os.makedirs(os.path.join(BASE, "reports"), exist_ok=True)

# IQVIA mock data (market)
iqvia_mock = [
    {"molecule": "Metformin", "therapy": "Diabetes", "market_size_usd_b": 4.5, "cagr_percent": 6.1,
     "top_competitors": ["Cipla", "Lupin", "Sun Pharma"]},
    {"molecule": "Aspirin", "therapy": "Cardiovascular", "market_size_usd_b": 2.1, "cagr_percent": 2.0,
     "top_competitors": ["Bayer", "Cipla"]}
]
with open(os.path.join(BASE, "mock_data", "iqvia_mock.json"), "w") as f:
    json.dump(iqvia_mock, f, indent=2)

# Patent mock
patent_mock = [
    {"molecule": "Metformin", "patents": [
        {"id": "US12345", "title": "Metformin for Diabetes", "expiry_year": 2028, "status": "Active"},
        {"id": "US67890", "title": "Metformin use in oncology", "expiry_year": 2022, "status": "Expired"}
    ]},
    {"molecule": "Aspirin", "patents": [
        {"id": "US54321", "title": "Aspirin formulation", "expiry_year": 2030, "status": "Active"}
    ]}
]
with open(os.path.join(BASE, "mock_data", "patent_mock.json"), "w") as f:
    json.dump(patent_mock, f, indent=2)

# Clinical trials mock
clinical_mock = [
    {"molecule": "Metformin", "trials": [
        {"id": "CT-001", "phase": "Phase II", "condition": "Obesity", "status": "Recruiting", "sponsor": "Uni A"},
        {"id": "CT-002", "phase": "Phase III", "condition": "PCOS", "status": "Completed", "sponsor": "Uni B"}
    ]},
    {"molecule": "Aspirin", "trials": [
        {"id": "CT-003", "phase": "Phase IV", "condition": "Cardiovascular", "status": "Active", "sponsor": "PharmaX"}
    ]}
]
with open(os.path.join(BASE, "mock_data", "clinical_mock.json"), "w") as f:
    json.dump(clinical_mock, f, indent=2)

# EXIM (CSV)
exim_rows = [
    ["molecule", "year", "export_value_usd_m", "import_value_usd_m", "top_dest_country"],
    ["Metformin", "2023", "50", "10", "USA"],
    ["Metformin", "2022", "45", "8", "USA"],
    ["Aspirin", "2023", "30", "12", "GER"]
]
with open(os.path.join(BASE, "mock_data", "exim_mock.csv"), "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(exim_rows)

# Web mock (scientific summaries)
web_mock = [
    {"molecule": "Metformin", "articles": [
        {"title": "Metformin shows promise in cancer models", "source": "Journal A", "year": 2023,
         "summary": "Preclinical studies show anti-proliferative effects."},
        {"title": "Metformin and longevity", "source": "Journal B", "year": 2022,
         "summary": "Association studies indicate potential life extension benefits."}
    ]},
    {"molecule": "Aspirin", "articles": [
        {"title": "Aspirin low-dose study", "source": "Journal C", "year": 2021, "summary": "Cardio-protective effects observed."}
    ]}
]
with open(os.path.join(BASE, "mock_data", "web_mock.json"), "w") as f:
    json.dump(web_mock, f, indent=2)

print("Mock data created at:", os.path.join(BASE, "mock_data"))


Mock data created at: /content/pharma_mas/mock_data


In [5]:
# Colab cell 4: force use of fallback classes to bypass CrewAI/OpenAI
USE_CREWAI = False

# Fallback: Implement minimal Agent/Task/Crew classes for orchestration
print("CrewAI not available; using local fallback agents as requested.")
from concurrent.futures import ThreadPoolExecutor, as_completed

# Minimal local Agent representation (for clarity)
class Agent:
    def __init__(self, role, goal, backstory=None, verbose=False):
        self.role = role
        self.goal = goal
        self.backstory = backstory
        self.verbose = verbose
    def run(self, input_data):
        raise NotImplementedError

class Task:
    def __init__(self, description, agent, func=None, expected_output=None, name=None):
        self.description = description
        self.agent = agent
        self.func = func  # optional function to call
        self.expected_output = expected_output # CrewAI compatibility
        self.name = name if name else description # Use name if provided, else description

class Crew:
    def __init__(self, agents, tasks, verbose=False):
        self.agents = agents
        self.tasks = tasks
        self.verbose = verbose
    def run(self, input_data):
        # run each task with a threadpool; tasks have agent.func to execute
        results = {}
        with ThreadPoolExecutor(max_workers=len(self.tasks)) as ex:
            futures = {}
            for t in self.tasks:
                if t.func is None:
                    continue
                # For fallback, we'll store results by task name
                futures[ex.submit(t.func, input_data)] = t.name
            for fut in as_completed(futures):
                task_name = futures[fut]
                try:
                    results[task_name] = fut.result()
                except Exception as e:
                    results[task_name] = {"error": str(e)}
        return results


CrewAI not available; using local fallback agents as requested.


In [6]:
# Colab cell 5: worker functions that act like Worker Agents
import json, pandas as pd
DATA_DIR = os.path.join(BASE, "mock_data")

def market_worker(input_data):
    molecule = input_data.get("molecule")
    with open(os.path.join(DATA_DIR, "iqvia_mock.json")) as f:
        iq = json.load(f)
    recs = [r for r in iq if r["molecule"].lower() == molecule.lower()]
    if not recs:
        return {"market": None, "note": "No market mock data found"}
    return {"market": recs[0]}

def patent_worker(input_data):
    molecule = input_data.get("molecule")
    with open(os.path.join(DATA_DIR, "patent_mock.json")) as f:
        pt = json.load(f)
    recs = [r for r in pt if r["molecule"].lower() == molecule.lower()]
    if not recs:
        return {"patents": [], "note": "No patent mock data found"}
    return {"patents": recs[0]["patents"]}

def clinical_worker(input_data):
    molecule = input_data.get("molecule")
    with open(os.path.join(DATA_DIR, "clinical_mock.json")) as f:
        ct = json.load(f)
    recs = [r for r in ct if r["molecule"].lower() == molecule.lower()]
    if not recs:
        return {"trials": [], "note": "No clinical mock data found"}
    return {"trials": recs[0]["trials"]}

def exim_worker(input_data):
    molecule = input_data.get("molecule")
    df = pd.read_csv(os.path.join(DATA_DIR, "exim_mock.csv"))
    md = df[df["molecule"].str.lower() == molecule.lower()]
    if md.empty:
        return {"exim": [], "note": "No EXIM mock data found"}
    # aggregate by year
    rows = md.to_dict(orient="records")
    return {"exim": rows}

def web_worker(input_data):
    molecule = input_data.get("molecule")
    with open(os.path.join(DATA_DIR, "web_mock.json")) as f:
        web = json.load(f)
    recs = [r for r in web if r["molecule"].lower() == molecule.lower()]
    if not recs:
        return {"articles": [], "note": "No web mock data found"}
    return {"articles": recs[0]["articles"]}

def internal_worker(input_data):
    # Simulated internal doc summary (we have no real PDFs here) - return a static insight to mimic parsing
    molecule = input_data.get("molecule")
    insights = {
        "Metformin": "Internal field reports indicate interest in metabolic effects and off-label research collaboration.",
        "Aspirin": "Internal sales teams report stable volumes for cardiovascular segment."
    }
    return {"internal_insight": insights.get(molecule, "No internal insight found")}


In [7]:
# Colab cell 6: construct agents/tasks and the crew
# Create Agent objects (either real CrewAI Agent or fallback Agent)
market_agent = Agent(role="Market Analyst", goal="Analyze market mock data", backstory="Expert in analyzing market trends and data.", verbose=True)
patent_agent = Agent(role="Patent Specialist", goal="Summarize patent mock data", backstory="Skilled in reviewing and summarizing patent information.", verbose=True)
clinical_agent = Agent(role="Clinical Trials Analyst", goal="Summarize clinical trials", backstory="Experienced in analyzing clinical trial data and results.", verbose=True)
exim_agent = Agent(role="Trade Analyst", goal="Summarize trade data", backstory="Knowledgeable in import and export data analysis.", verbose=True)
web_agent = Agent(role="Web Intelligence", goal="Summarize web articles and signals", backstory="Adept at gathering and summarizing information from the web.", verbose=True)
internal_agent = Agent(role="Internal Knowledge", goal="Summarize internal docs", backstory="Accesses and summarizes internal company documents and insights.", verbose=True)

# Build Tasks - each Task holds a description, agent, and function to run
market_task = Task(name="market", description="Analyze the provided market data for the given molecule and summarize key findings including market size, growth rate, and top competitors.", agent=market_agent, func=market_worker, expected_output="A summary of the market data for the specified molecule, including market size, CAGR, and key competitors.")
patent_task = Task(name="patents", description="Review and summarize the patent data for the given molecule, noting active and expired patents and their key details.", agent=patent_agent, func=patent_worker, expected_output="A summary of the patent information for the molecule, detailing active/expired status and relevant details for each patent.")
clinical_task = Task(name="clinical", description="Analyze the clinical trial data for the given molecule and provide a summary of ongoing and completed trials, including phase, condition, and sponsor.", agent=clinical_agent, func=clinical_worker, expected_output="A summary of clinical trials for the molecule, including phases, conditions, and sponsors.")
exim_task = Task(name="exim", description="Review the export and import data for the given molecule and summarize the key trade flows and top destinations.", agent=exim_agent, func=exim_worker, expected_output="A summary of export and import data for the molecule, including trade values and top destination countries.")
web_task = Task(name="web", description="Gather and summarize key insights from web articles and scientific summaries related to the given molecule.", agent=web_agent, func=web_worker, expected_output="A summary of key findings and insights from web articles and scientific literature about the molecule.")
internal_task = Task(name="internal", description="Summarize any relevant internal company insights or documents related to the given molecule.", agent=internal_agent, func=internal_worker, expected_output="A summary of internal company knowledge or insights pertaining to the molecule.")

# Assemble the Crew
pharma_crew = Crew(
    agents=[market_agent, patent_agent, clinical_agent, exim_agent, web_agent, internal_agent],
    tasks=[market_task, patent_task, clinical_task, exim_task, web_task, internal_task],
    verbose=True
)

print("Crew and tasks ready.")

Crew and tasks ready.


In [8]:
# Colab cell 7: Master Agent - orchestrates run and synthesizes summary
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import pprint

def master_run(molecule):
    input_data = {"molecule": molecule}
    # Run crew (which runs all task functions concurrently in fallback)
    # Use kickoff for CrewAI
    if USE_CREWAI:
        results = pharma_crew.kickoff(inputs=input_data)
    else:
        results = pharma_crew.run(input_data)


    # results keys are task descriptions; unify into structured summary
    summary = {"molecule": molecule, "timestamp": datetime.utcnow().isoformat()}
    # map known task keys
    # Note: CrewAI kickoff returns a single string output, not a dictionary of task results like the fallback
    # We need to parse the CrewAI output or adjust the fallback for consistency if needed for further processing.
    # For now, let's just include the raw output from CrewAI or the structured results from fallback.

    if USE_CREWAI:
        summary["raw_crew_output"] = results
        # If further structured processing is needed, we would need an agent to parse the 'results' string
        # and extract structured information.
    else:
        # Access results using the new 'name' attribute of the tasks
        summary["market"] = results.get("market", {})
        summary["patents"] = results.get("patents", {})
        summary["clinical"] = results.get("clinical", {})
        summary["exim"] = results.get("exim", {})
        summary["web"] = results.get("web", {})
        summary["internal"] = results.get("internal", {})

        # Simple heuristic to flag repurposing opportunity (only applies to fallback structured output):
        # - If clinical trials exist outside main therapy (e.g., Metformin in cancer), or
        # - If patents expired and market size indicates opportunity.
        flags = []
        try:
            # check web articles for off-label signals
            articles = summary["web"].get("articles", [])
            off_label_hits = [a for a in articles if any(word in a["title"].lower() for word in ["cancer","longevity","aging","oncology"])]
            if off_label_hits:
                flags.append("Off-label scientific signal (web articles)")
        except Exception:
            pass
        try:
            patents = summary["patents"].get("patents", [])
            expired = [p for p in patents if p.get("status"," confined").lower() == "expired" or p.get("expiry_year",9999) <= datetime.utcnow().year]
            if expired:
                flags.append("Expired patents — potential freedom to operate")
        except Exception:
            flags.append("Error processing patent data for flags")
        try:
            market = summary["market"].get("market")
            if market and market.get("market_size_usd_b",0) > 1:
                flags.append("Large therapy market (> $1B) — commercial interest")
        except Exception:
            flags.append("Error processing market data for flags")

        summary["repurposing_flags"] = flags or ["No strong flags from mock data"]


    return summary

# Test run
demo = master_run("Metformin")
pprint.pprint(demo)

{'clinical': {'trials': [{'condition': 'Obesity',
                          'id': 'CT-001',
                          'phase': 'Phase II',
                          'sponsor': 'Uni A',
                          'status': 'Recruiting'},
                         {'condition': 'PCOS',
                          'id': 'CT-002',
                          'phase': 'Phase III',
                          'sponsor': 'Uni B',
                          'status': 'Completed'}]},
 'exim': {'exim': [{'export_value_usd_m': 50,
                    'import_value_usd_m': 10,
                    'molecule': 'Metformin',
                    'top_dest_country': 'USA',
                    'year': 2023},
                   {'export_value_usd_m': 45,
                    'import_value_usd_m': 8,
                    'molecule': 'Metformin',
                    'top_dest_country': 'USA',
                    'year': 2022}]},
 'internal': {'internal_insight': 'Internal field reports indicate interest in '
         

/tmp/ipython-input-2161534846.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  summary = {"molecule": molecule, "timestamp": datetime.utcnow().isoformat()}
/tmp/ipython-input-2161534846.py:50: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  expired = [p for p in patents if p.get("status"," confined").lower() == "expired" or p.get("expiry_year",9999) <= datetime.utcnow().year]


In [9]:
# Colab cell 8: generate a PDF report and an Excel summary from the synthesized summary
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table
from reportlab.lib.styles import getSampleStyleSheet
import pandas as pd
import json

def generate_pdf_report(summary, out_path):
    styles = getSampleStyleSheet()
    doc = SimpleDocTemplate(out_path)
    story = []
    story.append(Paragraph(f"Drug Repurposing Report - {summary['molecule']}", styles["Heading1"]))
    story.append(Paragraph(f"Generated: {summary['timestamp']}", styles["Normal"]))
    story.append(Spacer(1,12))
    # Market
    market = summary.get("market", {}).get("market")
    story.append(Paragraph("Market Summary:", styles["Heading2"]))
    if market:
        md = [
            ["Therapy", market.get("therapy")],
            ["Market Size (USD B)", str(market.get("market_size_usd_b"))],
            ["CAGR (%)", str(market.get("cagr_percent"))],
            ["Top Competitors", ", ".join(market.get("top_competitors", []))]
        ]
        story.append(Table(md))
    else:
        story.append(Paragraph("No market data available.", styles["Normal"]))
    story.append(Spacer(1,12))

    # Patents
    story.append(Paragraph("Patent Summary:", styles["Heading2"]))
    patents = summary.get("patents", {}).get("patents", [])
    if patents:
        p_rows = [["Patent ID", "Title", "Expiry Year", "Status"]]
        for p in patents:
            p_rows.append([p.get("id"), p.get("title"), str(p.get("expiry_year")), p.get("status")])
        story.append(Table(p_rows))
    else:
        story.append(Paragraph("No patents found.", styles["Normal"]))
    story.append(Spacer(1,12))

    # Clinical
    story.append(Paragraph("Clinical Trials Summary:", styles["Heading2"]))
    trials = summary.get("clinical", {}).get("trials", [])
    if trials:
        t_rows = [["Trial ID", "Phase", "Condition", "Status", "Sponsor"]]
        for t in trials:
            t_rows.append([t.get("id"), t.get("phase"), t.get("condition"), t.get("status"), t.get("sponsor")])
        story.append(Table(t_rows))
    else:
        story.append(Paragraph("No clinical trial data found.", styles["Normal"]))
    story.append(Spacer(1,12))

    # Web + internal insights + flags
    story.append(Paragraph("Web Insights (top article titles):", styles["Heading2"]))
    articles = summary.get("web", {}).get("articles", [])
    if articles:
        for a in articles:
            story.append(Paragraph(f"- {a['title']} ({a['source']}, {a['year']}): {a['summary']}", styles["Normal"]))
    else:
        story.append(Paragraph("No web articles found.", styles["Normal"]))
    story.append(Spacer(1,12))

    story.append(Paragraph("Internal Insight:", styles["Heading2"]))
    story.append(Paragraph(summary.get("internal", {}).get("internal_insight", "No internal insight"), styles["Normal"]))
    story.append(Spacer(1,12))

    story.append(Paragraph("Repurposing Flags / Heuristics:", styles["Heading2"]))
    for f in summary.get("repurposing_flags", []):
        story.append(Paragraph(f"- {f}", styles["Normal"]))

    doc.build(story)
    return out_path

def generate_excel_summary(summary, out_path):
    # Flatten the summary into multiple sheets
    writer = pd.ExcelWriter(out_path, engine="openpyxl")
    # Market
    market = summary.get("market", {}).get("market")
    if market:
        pd.DataFrame([market]).to_excel(writer, sheet_name="Market", index=False)
    else:
        pd.DataFrame([{}]).to_excel(writer, sheet_name="Market", index=False)
    # Patents
    patents = summary.get("patents", {}).get("patents", [])
    pd.DataFrame(patents).to_excel(writer, sheet_name="Patents", index=False)
    # Clinical
    trials = summary.get("clinical", {}).get("trials", [])
    pd.DataFrame(trials).to_excel(writer, sheet_name="Clinical", index=False)
    # EXIM
    exim = summary.get("exim", {}).get("exim", [])
    pd.DataFrame(exim).to_excel(writer, sheet_name="EXIM", index=False)
    # Web
    articles = summary.get("web", {}).get("articles", [])
    pd.DataFrame(articles).to_excel(writer, sheet_name="Web", index=False)
    # Flags & internal
    pd.DataFrame({"flags": summary.get("repurposing_flags", [])}).to_excel(writer, sheet_name="Flags", index=False)
    pd.DataFrame([{"internal": summary.get("internal", {}).get("internal_insight", "")}]).to_excel(writer, sheet_name="Internal", index=False)

    writer.close()
    return out_path

# Generate files for demo summary
demo_summary = demo  # from earlier master_run example
pdf_path = os.path.join(BASE, "reports", f"{demo_summary['molecule']}_Report.pdf")
xlsx_path = os.path.join(BASE, "reports", f"{demo_summary['molecule']}_Summary.xlsx")

generate_pdf_report(demo_summary, pdf_path)
generate_excel_summary(demo_summary, xlsx_path)
print("Report files created:")
print(pdf_path)
print(xlsx_path)


Report files created:
/content/pharma_mas/reports/Metformin_Report.pdf
/content/pharma_mas/reports/Metformin_Summary.xlsx


In [10]:
# Colab cell 9: show final summary and provide download link (Colab environment)
import json, os
from IPython.display import FileLink, display

# Show summary
print("Final summary (pretty):")
import pprint
pprint.pprint(demo_summary)

# Provide download links to the generated files
display(FileLink(pdf_path))
display(FileLink(xlsx_path))


Final summary (pretty):
{'clinical': {'trials': [{'condition': 'Obesity',
                          'id': 'CT-001',
                          'phase': 'Phase II',
                          'sponsor': 'Uni A',
                          'status': 'Recruiting'},
                         {'condition': 'PCOS',
                          'id': 'CT-002',
                          'phase': 'Phase III',
                          'sponsor': 'Uni B',
                          'status': 'Completed'}]},
 'exim': {'exim': [{'export_value_usd_m': 50,
                    'import_value_usd_m': 10,
                    'molecule': 'Metformin',
                    'top_dest_country': 'USA',
                    'year': 2023},
                   {'export_value_usd_m': 45,
                    'import_value_usd_m': 8,
                    'molecule': 'Metformin',
                    'top_dest_country': 'USA',
                    'year': 2022}]},
 'internal': {'internal_insight': 'Internal field reports indicate

/content/pharma_mas/reports/Metformin_Report.pdf

/content/pharma_mas/reports/Metformin_Summary.xlsx

In [10]:
from google.colab import files

# Trigger download for the PDF file
files.download(pdf_path)
print(f"Downloading {pdf_path}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [57]:
from google.colab import files

# Trigger download for the Excel file
files.download(xlsx_path)
print(f"Downloading {xlsx_path}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb


--2025-12-15 18:07:58--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb [following]
--2025-12-15 18:07:58--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/8a32f7c6-649c-4f0d-806d-e14c19d0786d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-15T18%3A52%3A06Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4d

In [37]:
!sudo dpkg -i cloudflared-linux-amd64.deb


Selecting previously unselected package cloudflared.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.11.1) ...
Setting up cloudflared (2025.11.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [38]:
!which cloudflared


/usr/local/bin/cloudflared


In [28]:
!pip install streamlit


In [32]:
%%writefile /content/pharma_mas/app.py
import streamlit as st
import os

st.title("💊 PharmaMAS — Multi-Agent System (Mock Demo)")

molecule = st.text_input("Enter Molecule", "Metformin")

if st.button("Run Analysis"):
    st.success(f"Running MAS for: {molecule}")

    st.write("📄 Report files will be located here:")
    st.code("/content/pharma_mas/reports/")

    st.info("Go to the Colab **Files** panel → open `pharma_mas/reports/` → download your PDF & Excel.")


Overwriting /content/pharma_mas/app.py


In [39]:
!streamlit run /content/pharma_mas/app.py --server.port 8501 --server.address 0.0.0.0 &>/content/app.log &


In [40]:
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate


2025-12-15T18:09:16Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-12-15T18:09:16Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-12-15T18:09:19Z INF +--------------------------------------------------------------------------------------------+
2025-12-15T18:09:19Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-12-15T18:09:19Z INF |  https://impose-supported-portal-version.trycloudflare